# EE4375-2022: Fifth Lab Session: One-Dimensional Galerkin Finite Element Method using Distributed Memory Parallel Computing

Solves the Poisson equation $- \frac{d^2 \, u(x)}{dx^2} = f(x)$ on the unit bar domain $x \in \Omega=(0,1)$ supplied with various boundary conditions and various source terms. The Galerkin finite element method is employed. Here we target a parallel computing (using distributed memory) imnplementation. 

This problem can be solved using [GridapDistributed](https://gridap.github.io/GridapDistributed.jl/dev/) as students in the bachelor minor Computational Science and Engineering have convincingly shown. It remains valuable to dig for the details. 

General info on [parallel computing in Julia](https://juliaparallel.org/resources/) and [MPI.jl](https://github.com/JuliaParallel/MPI.jl). 

## Import Packages

In [9]:
using LinearAlgebra
using Plots
using LaTeXStrings
@everywhere using SparseArrays
using BenchmarkTools
using Distributed
@everywhere using SharedArrays

## Section 1: Preprocessing 
- <b> shared memory approach </b>: monolytic in memory approach: assuming global mesh, matrix and rhs vector to be available on all processors: assembly process similar to all processors; 
- <b> distributed memory approach </b>: distribute memory approach: distribute memory approach: distributed assembly and solve; include figure here; 

## Section 2: Shared Memory Assembly of Matrix and Right-Hand Side Vector 
- <b> decompose mesh elements over the processors </b>: observe mesh to be decomposed according to elements and <b>not</b> according to nodes; 
- <b> use shared memory for-loop over elements </b>: the for-loop over elements can run in parallel in shared memory using either the macro @distributed or pmap function from the  [distributed computing section](https://docs.julialang.org/en/v1/stdlib/Distributed/) of the standard library; 
- <b> used SharedVector and SharedMatrix to store answers</b>: using SharedVector and SharedMatrix using [shared arrays](https://docs.julialang.org/en/v1/stdlib/SharedArrays/); each processor fills its part of the matrix; 
- not sure whether [Distributed Arrays](https://juliaparallel.org/DistributedArrays.jl/stable/) offers any advantages; 
- valuable example on the use of SharedArray is [sharedarrays-and-data-movement-across-processes-in-julia](https://stackoverflow.com/questions/53753789/sharedarrays-and-data-movement-across-processes-in-julia)
- a schematic representation of the shared memory implementation would be valuable to have; 

### Documentation on the following would be valuable to add 

In [10]:
nprocs()

5

In [11]:
# small example on how to initialize a shared vector and matrix. 
# observe the matrix to be dense.  
f = SharedVector{Float64}(5)
A = SharedArray{Float64}((5,5))
procs(A) # what does this do? 
localindices(A) # what does this do? 
indexpids(A) # what does this do? 

0

### Shared Memory Implementation of 1D Galerkin FEM: Option-1 
The implementation of Option-1 employs SharedArrays to store the coefficient matrix. This implementation requires storing the coefficient matrix as a <b>dense</b> matrix. It is thus inadequate. 

In [7]:
# start REPL using julia -p 5 
addprocs(4)
no_procs = nprocs()

#..construct the mesh: see before 
@everywhere N = 100; 
@everywhere h = 1/N; 
@everywhere x = Vector(0:h:1); 

#..Mesh with points and edges 
#..point holds the coordinates of the left and right node of the element
#..edges holds the global indices of the left and right node of the element
@everywhere points = collect( [x[i], x[i+1]] for i in 1:length(x)-1) 
@everywhere edges = collect( [i, i+1] for i in 1:length(x)-1); 

#..Set the source function 
@everywhere fsource(x) = x*(x-1); 

#..Initialize global matrix and right-hand side value 
A = SharedArray{Float64}((length(x), length(x))); # type annotation required here 
f = SharedVector{Float64}(length(x)); # type annotation required here 

#..Perform loop over elements and assemble global matrix and vector 
@distributed for i=1:length(edges) 
  xl, xr = points[i,:][1]
  floc = (xr-xl) * [fsource(xl); fsource(xr)];
  Aloc = (1/(xr-xl))*[1 -1; -1 1]; 
  f[edges[i]] += floc
  A[edges[i], edges[i]] += Aloc
end

Task (runnable) @0x00000002898f7540

### Shared Memory Implementation of 1D Galerkin FEM: Option-2 
The implementation of Option-2 employs sparse matrix to store the coefficient matrix. All threads see the complete matrix. This implementation does not pre-allocate the memory of the sparse matrix and is thus inadequate. 

In [ ]:
# start REPL using julia -p 5 
addprocs(4)
no_procs = nprocs()

#..construct the mesh: see before 
@everywhere N = 100; 
@everywhere h = 1/N; 
@everywhere x = Vector(0:h:1); 

#..Mesh with points and edges 
#..point holds the coordinates of the left and right node of the element
#..edges holds the global indices of the left and right node of the element
@everywhere points = collect( [x[i], x[i+1]] for i in 1:length(x)-1) 
@everywhere edges = collect( [i, i+1] for i in 1:length(x)-1); 

#..Set the source function 
@everywhere fsource(x) = x*(x-1); 

#..Initialize global matrix and right-hand side value 
@everywhere A = spzeros(Float64,length(x),length(x))
@everywhere f = zeros(Float64,length(x), 1)

#..Perform loop over elements and assemble global matrix and vector 
@distributed for i=1:length(edges) 
  xl, xr = points[i,:][1]
  floc = (xr-xl) * [fsource(xl); fsource(xr)];
  Aloc = (1/(xr-xl))*[1 -1; -1 1]; 
  f[edges[i]] += floc
  A[edges[i], edges[i]] += Aloc
end

### Shared Memory Implementation of 1D Galerkin FEM: Option-3 (in progress) 
The implementation of Option-3 employs the sparse command sequentially on three lists generated in parallel. 

In [ ]:
# struct to hold single element
struct Element
  p1::Float64
  p2::Float64
  e1::Int64
  e2::Int64
end 

function fem_1d_sparse(N)
    
    if (Bool(0)) print(" [fem_1d]:: input N = ", N, "\n") end 
    
    #..Generate the mesh 
    Np1 = N+1; h = 1/N
    x = Vector(0:h:1) 
    mesh = StructArray{Element}((x[1:end-1], x[2:end], Vector(1:N), Vector(2:N+1)))

    if (Bool(0)) print(" [fem_1d]:: mesh = ", mesh, "\n") end

    #..Set the source function 
    fsource(x) = x*(x-1)

    #..Initialize global matrix and right-hand side value 
    f = zeros(Float64,Np1,1)
    I = zeros(Int64,4*N)
    J = zeros(Int64,4*N)
    Avalues = zeros(Float64,4*N)
    floc = zeros(Float64,2, 1)
    Aloc = zeros(Float64,2,2)

    #..Perform loop over elements and assemble global matrix and vector 
    @inbounds for i=1:N 
        
      xl = mesh[i].p1
      xr = mesh[i].p2
      j  = mesh[i].e1
      k  = mesh[i].e2
        
      floc = (xr-xl) * [fsource(xl), fsource(xr)];
      Aloc = (1/(xr-xl))*[1, -1, -1, 1]; 
        
      f[[j,k]] += floc 
      I[4*(i-1)+1:4*i] = [j, k, j, k]
      J[4*(i-1)+1:4*i] = [j, j, k, k]
      Avalues[4*(i-1)+1:4*i] = Aloc 
        
    end

    A = sparse(I,J,Avalues)

    #..handle the boundary conditions in the matrix and right-hand side vector 
    A[1,1] = 1;     A[1,2] = 0;        f[1]   = 0
    A[end,end-1]=0; A[end,end] = 1;    f[end] = 0

    #..solve the linear system
    u = A \ f  
    
    return x, u 
    
end

### Questions 
1. does increasing number of processors decrease overall CPU time? 
2. can @distributed for be replace by pmap? See [this link](https://jishnub.github.io/ParallelUtilities.jl/dev/examples/sharedarrays/). Does it offer advantages? 
3. do sparse distributed arrays exist? How to perform memory allocations of these arrays?

## Section 3: Distributed Memory Assembly of Matrix and Right-Hand Side Vector 
- using [PartionedMatrices](https://github.com/fverdugo/PartitionedArrays.jl); need to study the example described at [example](https://www.francescverdugo.com/PartitionedArrays.jl/dev/usage/).  

<div>
<img src="./figures/dag-fem-1d.jpg" width=400 /> 
<center> Figure 1: Directed acyclic graph representation of decomposed mesh of 4 elements on the interval. A finite element discretization is assumed. </center>   
</div>

## Section 4: Shared Memory Linear System Solve 
- <b> first approach </b>: use backslash (seems to works, not clear what is does internally); 
- <b> second approach </b>: use packages such as [Pardiso.jl](https://github.com/JuliaSparse/Pardiso.jl) or [SuiteSparseGraphBLAS.jl](https://docs.juliahub.com/SuiteSparseGraphBLAS/HtDaW/0.6.0/#Introduction). 
- <b> third approach </b>: use home-brewed [preconditioned conjugate gradient method](https://en.wikipedia.org/wiki/Conjugate_gradient_method). Sequential version is available from [IterativeSolvers.jl]([https://github.com/JuliaLinearAlgebra/IterativeSolvers.jl). Use parallel BLAS1 and BLAS2 functions; using [sparse-matrix multiplication](https://github.com/JuliaInv/ParSpMatVec.jl);

### Parallel BLAS-1 Operations 
Using [ParallelUtilities](https://docs.juliahub.com/ParallelUtilities/SO4iL/0.7.0/). 

#### Vector Norm 
Given vector ${\mathbf v}$, compute its squared norm $\| \mathbf v \|^2$, using pmapsum(x->x^2,v). Alternatively, this is the function dot with the same input argument repeated.  

#### Vectors Inner Product 
Given vectors ${\mathbf v}$ and ${\mathbf w}$, compute their inner product ${\mathbf v} \cdot {\mathbf w}$, using pmapsum((x,y)->x*y,v,w). See [this post](https://discourse.julialang.org/t/function-pmap-multi-argument/74153) for an example of pmap using two arguments. Alternatively, this is the function dot. 
 
#### Vector Update 
Given vectors ${\mathbf v}$ and ${\mathbf w}$, and given a number $\alpha$ (typically the result of an inner product evaluation), compute the vector update ${\mathbf w} = {\mathbf w} + \alpha {\mathbf v}$, using pmapsum((x,y,a)->y+a*x,v,w,alpha). Alternatively, we can update the components of the vector ${\mathbf w}$ elementwise. 

### Parallel BLAS-2 Operations 

#### Sparse Matrix-Vector Multiplication 
Use [ParSpMatVec](https://github.com/JuliaInv/ParSpMatVec.jl/blob/master/test/test_A_mul_B.jl) for sparse matrix vector multiplication. Alternatively, this is the function mul!. 
Background information is provided in e.g. [Parallel Linear Algebra by Deprez](https://fdesprez.github.io/teaching/par-comput/lectures/slides/L8-AlLinPar-2p.pdf); 

### Steepest Descent Method 
Combine ingredients above to implement the steepest descent method as described in Section [Solution to a Linear System](https://en.wikipedia.org/wiki/Gradient_descent#Solution_of_a_linear_system). 

### Conjugate Gradient Method 
Extend steepest descent to the conjugate gradient method as described in Section [Resulting Algorithm](https://en.wikipedia.org/wiki/Conjugate_gradient_method#The_resulting_algorithm). An explanation of the CG algorithm using iterates in explained on [iterative-methods-done-right](https://lostella.github.io/2018/07/25/iterative-methods-done-right.html). Note use PCG with proper [overlap of computation and communication](https://netlib.org/linalg/html_templates/node107.html#SECTION00941100000000000000);  

 

## Section 5: Distributed Memory Linear System Solve 
Using the PETSc library (as done by the bachelor students). 

## Section 6: Postprocessing 
Visualize the computed solution. 